# 期中考
***
Kaggle期中考知識點⽬標
競賽結束後你可以學會
- 如何處理存在各種缺陷的真實資料
- 使⽤ val / test data 來了解機器學習模型的訓練情形
- 使⽤適當的評估函數了解預測結果
- 應⽤適當的特徵⼯程提升模型的準確率
- 調整機器學習模型的超參數來提升準確率
-清楚的說明⽂件讓別⼈了解你的成果

https://www.kaggle.com/c/3rd-ml100marathon-midterm/data?select=train_data.csv

In [45]:
import copy, time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [7]:
data_path = './'
df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')

In [8]:
set(df_train.columns) - set(df_test.columns)

{'poi'}

In [9]:
df_train.head()

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [10]:
df_test.head()

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,BELDEN TIMOTHY N,5249999.0,2144013.0,-2334434.0,NaN,tim.belden@enron.com,953136.0,17355.0,484.0,228.0,...,NaN,NaN,210698.0,157569.0,NaN,213999.0,5521.0,7991.0,5501630.0,1110705.0
1,BOWEN JR RAYMOND M,1350000.0,NaN,-833.0,NaN,raymond.bowen@enron.com,NaN,65907.0,27.0,140.0,...,NaN,974293.0,1621.0,252055.0,NaN,278601.0,1593.0,1858.0,2669589.0,252055.0
2,HANNON KEVIN P,1500000.0,NaN,-3117011.0,NaN,kevin.hannon@enron.com,5538001.0,34039.0,32.0,32.0,...,NaN,1617011.0,11350.0,853064.0,NaN,243293.0,1035.0,1045.0,288682.0,6391065.0
3,DELAINEY DAVID W,3000000.0,NaN,NaN,NaN,david.delainey@enron.com,2291113.0,86174.0,3069.0,66.0,...,NaN,1294981.0,1661.0,1323148.0,NaN,365163.0,2097.0,3093.0,4747979.0,3614261.0
4,CAUSEY RICHARD A,1000000.0,NaN,-235000.0,NaN,richard.causey@enron.com,NaN,30674.0,49.0,58.0,...,NaN,350000.0,307895.0,2502063.0,NaN,415189.0,1585.0,1892.0,1868758.0,2502063.0


In [11]:
print(f'train.shape: {df_train.shape}')
print(f'df_test.shape: {df_test.shape}')

train.shape: (113, 22)
df_test.shape: (33, 21)


In [12]:
len(df_test['name'].unique())

33

In [13]:
train_Y = df_train['poi']
ids = df_test['name']
df_train = df_train.drop(['poi'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [14]:
print(f'df.shape: {df.shape}')

df.shape: (146, 21)


In [15]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(10))
na_check(df)

,Missing Ratio
loan_advances,97.260274
director_fees,88.356164
restricted_stock_deferred,87.671233
deferral_payments,73.287671
deferred_income,66.438356
long_term_incentive,54.794521
bonus,43.835616
from_messages,41.095890
from_poi_to_this_person,41.095890
from_this_person_to_poi,41.095890


In [16]:
obj_feature = []
int_feature = []
for dtype, feature in zip(df.dtypes, df.columns):
#     print(f'dtype: {dtype}, feature: {feature}')
    if dtype == 'float64' or dtype == 'int64':
        int_feature.append(feature)
    else:
        obj_feature.append(feature)

print(f'obj feature:{obj_feature}')
print(f'int feature:{int_feature}')

obj feature:['name', 'email_address']
int feature:['bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']


In [17]:
# 缺失捕0
for col in int_feature:
    df[col] = df[col].fillna(0)

In [18]:
na_check(df)

,Missing Ratio
email_address,23.972603


In [20]:
df['email_address'] = df['email_address'].fillna('None')

In [21]:
na_check(df)

,Missing Ratio


In [22]:
df = pd.get_dummies(df)

In [23]:
df.shape

(146, 277)

In [26]:
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

In [29]:
test_X.head()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,...,email_address_terence.thorn@enron.com,email_address_thomas.white@enron.com,email_address_tim.belden@enron.com,email_address_timothy.detmering@enron.com,email_address_tod.lindholm@enron.com,email_address_tracy.foy@enron.com,email_address_vicki.sharp@enron.com,email_address_vince.kaminski@enron.com,email_address_w.duran@enron.com,email_address_wes.colwell@enron.com
0,5249999.0,2144013.0,-2334434.0,0.0,953136.0,17355.0,484.0,228.0,108.0,0.0,...,0,0,1,0,0,0,0,0,0,0
1,1350000.0,0.0,-833.0,0.0,0.0,65907.0,27.0,140.0,15.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1500000.0,0.0,-3117011.0,0.0,5538001.0,34039.0,32.0,32.0,21.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3000000.0,0.0,0.0,0.0,2291113.0,86174.0,3069.0,66.0,609.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1000000.0,0.0,-235000.0,0.0,0.0,30674.0,49.0,58.0,12.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
train_Y

0       True
1       True
2       True
3       True
4       True
5       True
6       True
7       True
8       True
9       True
10      True
11      True
12      True
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
83     False
84     False
85     False
86     False
87     False
88     False
89     False
90     False
91     False
92     False
93     False
94     False
95     False
96     False
97     False
98     False
99     False
100    False
101    False
102    False
103    False
104    False
105    False
106    False
107    False
108    False
109    False
110    False
111    False
112    False
Name: poi, Length: 113, dtype: bool

In [52]:
# 羅輯回歸
lr = LogisticRegression()
lr.fit(train_X, train_Y)
lr_pred = lr.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': df_test['name'], 'poi': lr_pred})
sub.to_csv('3dr_100_lr.csv', index=False)

In [54]:
# 隨機森林
rf = RandomForestClassifier()
rf.fit(train_X, train_Y)
rf_pred = rf.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': df_test['name'], 'poi': rf_pred})
sub.to_csv('3dr_100_rf.csv', index=False)

In [55]:
# gdbt
gdbt = GradientBoostingClassifier()
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name': df_test['name'], 'poi': gdbt_pred})
sub.to_csv('3dr_100_gdbt.csv', index=False)

![成績](image.png)